## Decompose

In [147]:
from rdkit.Chem import BRICS
from rdkit import Chem
from rdkit.Chem import Draw
demo_smi='CN(C)c1nc2c(ncn2-c2ccc3ccccc3c2)c(=O)[nH]1'
# demo_smi='O=c1[nH]c(F)nc2c1NC(c1csc(-c3c[nH]c4ccccc34)n1)CN2'
# demo_smi='CC(=O)n1cc(-c2ccc3ccccc3c2)c(=O)[nH]c1=O'
# demo_smi='O=c1nc(Cl)ccn1-c1[nH]ncc1C1CC=CCC1'
m=Chem.MolFromSmiles(demo_smi)
smis=BRICS.BRICSDecompose(m) #silent
res_bond = list(BRICS.FindBRICSBonds(m))
sorted(res_bond),smis

([((1, 3), ('5', '14')), ((9, 10), ('9', '16'))],
 {'[16*]c1ccc2ccccc2c1', '[5*]N(C)C', '[9*]n1cnc2c(=O)[nH]c([14*])nc21'})

In [148]:
# smis=['[5*]N1CCC([15*])CC1']
import re
match_dict={}
order_dict={}

for sdx,smi in enumerate(smis):
    ori_smi=smi
    
    link_num_lst=re.findall("\[(.*?)\]",smi, re.I|re.M)

    new_link_num_lst=[]
    # for link_num in link_num_lst:
    #     if not 'H' in link_num:
    #         new_link_num_lst.append(link_num)
    for link_num in link_num_lst:
        if not 'H' in link_num and not '+' in link_num and not '-' in link_num:
            new_link_num_lst.append(link_num)
    link_num_lst=new_link_num_lst

    for link_num in link_num_lst:
        smi=smi.replace("["+link_num+"]", "*")
    print(smi)
    patt = Chem.MolFromSmarts(smi)
    match_idx=m.GetSubstructMatches(patt)
    # print(smi)
    # print(match_idx)
    match_dict[ori_smi]=match_idx
    
    order_dict[ori_smi]=sdx

*c1ccc2ccccc2c1
*N(C)C
*n1cnc2c(=O)[nH]c(*)nc21


In [149]:
match_dict

{'[16*]c1ccc2ccccc2c1': ((9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19),),
 '[5*]N(C)C': ((3, 1, 0, 2),),
 '[9*]n1cnc2c(=O)[nH]c([14*])nc21': ((10,
   9,
   8,
   7,
   6,
   20,
   21,
   22,
   3,
   1,
   4,
   5),)}

In [150]:
sorted(res_bond)

[((1, 3), ('5', '14')), ((9, 10), ('9', '16'))]

In [151]:
import re

def identify_free_bond_pos(smi,matched_atom_idxs,link_value,bond_lst):
    if len(matched_atom_idxs)>1:
        for atom_idxs in matched_atom_idxs:
            if sorted(list(set(atom_idxs).intersection(set(bond_lst))))==sorted(list(bond_lst)):
                unique_atom_idx=atom_idxs
    elif len(matched_atom_idxs)==1:
        unique_atom_idx=matched_atom_idxs[0]
    else:
        print("no match:{len(matched_atom_idxs)}")
        return None

    print(f"INPUT\nfrag smi:{smi};unique_atom_idx:{unique_atom_idx};link_value:{link_value};bond_lst:{bond_lst}")
    smi_char_lst=re.split('\[|\]|\(|\)',smi)
    new_smi_char_lst=[]

    num_lst=[str(i) for i in list(range(9))]
    for smi_char in smi_char_lst:

        if not smi_char=='':
            if '*' in smi_char:
                new_smi_char_lst.append(smi_char)
            else:
                for a in smi_char:
                    if not a in num_lst:
                        new_smi_char_lst.append(a)

    # print(f"new_smi_char_lst:")
    # print(new_smi_char_lst)
    # print(unique_atom_idx)

    mol=Chem.MolFromSmiles(smi)
    atoms_num=len(mol.GetAtoms())
    bond_pos=-1

    print(f"atom num==len(new_smi_char_lst):{atoms_num==len(new_smi_char_lst)}")
    n=0
    for cdx,atom in enumerate(mol.GetAtoms()):
        if (atom.GetSymbol() == "*"):
            if new_smi_char_lst[cdx]==link_value+"*":
                bond=unique_atom_idx[cdx]
                # print(bond)
                # print(bond_lst)
                if bond in bond_lst:
                    bond_pos=atom.GetIdx()
                    if not bond_pos==0:
                        bond_pos=bond_pos-n-1
                    print(f"matched bond_pos:{bond_pos}")
            n+=1
    if bond_pos==-1:
        print(f"\nplease check the order of atom index...")
        print(f"atom list:{new_smi_char_lst}; link index list:{unique_atom_idx}; link_bond:{bond_lst}\n")
    return bond_pos

In [152]:
keys=match_dict.keys()
smi_jbond=False
jbond_lst=[]
for tup in sorted(res_bond):
    bond_lst=tup[0]      # (2,1)
    idx_lst=tup[1]       # ('4','11')

    jbond=['','',-2,-2]    # [block_smi1, block_smi2,break_atom1,break_atom2]
    idx_dict={}          # key='4','11'   value=[frag_smis]

    for ldx,link_idx in enumerate(idx_lst):
        idx_dict[link_idx]=[]
        print("---------------------------------------------------")
        print(f"link_idx:{link_idx}")

        for key in keys:
            if "["+link_idx+"*]" in key:
                idx_dict[link_idx]+=[key]

        if len(idx_dict[link_idx])==1:
            frag_smi=idx_dict[link_idx][0]
            if smi_jbond:

                jbond[ldx]=frag_smi
            else:
                jbond[ldx]=order_dict[idx_dict[link_idx][0]]

            bond_pos=identify_free_bond_pos(frag_smi,match_dict[frag_smi],link_idx,bond_lst)
            jbond[ldx+2]=bond_pos #bond_lst[ldx]

        else:
            print(f"idx_dict[{link_idx}]:")
            print(idx_dict[link_idx])
            for match_smi in idx_dict[link_idx]:
                atom_idxs=match_dict[match_smi]
                #print(atom_idxs)
                if len(atom_idxs)==1:
                    if sorted(list(set(atom_idxs[0]).intersection(set(bond_lst))))==sorted(list(bond_lst)):
                        #print("True")
                        if smi_jbond:
                            jbond[ldx]=match_smi
                        else:
                            jbond[ldx]=order_dict[match_smi]
                        bond_pos=identify_free_bond_pos(match_smi,match_dict[match_smi],link_idx,bond_lst)
                        jbond[ldx+2]=bond_pos #bond_lst[ldx]
                        break
                else:
                    if jbond[ldx+2]==-2:
                        print("run multiple idx fragment...")
                        for possible_atom_idx in atom_idxs:
                            #print(possible_atom_idx)
                            if sorted(list(set(possible_atom_idx).intersection(set(bond_lst))))==sorted(list(bond_lst)):
                                #print("True")
                                if smi_jbond:
                                    jbond[ldx]=match_smi
                                else:
                                    jbond[ldx]=order_dict[match_smi]
                                bond_pos=identify_free_bond_pos(match_smi,[possible_atom_idx],link_idx,bond_lst)
                                jbond[ldx+2]=bond_pos #bond_lst[ldx]
                                break

    print("***********jbonds:****************")
    print(tup)
    print(jbond)
    jbond_lst.append(jbond)

---------------------------------------------------
link_idx:5
INPUT
frag smi:[5*]N(C)C;unique_atom_idx:(3, 1, 0, 2);link_value:5;bond_lst:(1, 3)
atom num==len(new_smi_char_lst):True
matched bond_pos:0
---------------------------------------------------
link_idx:14
INPUT
frag smi:[9*]n1cnc2c(=O)[nH]c([14*])nc21;unique_atom_idx:(10, 9, 8, 7, 6, 20, 21, 22, 3, 1, 4, 5);link_value:14;bond_lst:(1, 3)
atom num==len(new_smi_char_lst):False

please check the order of atom index...
atom list:['9*', 'n', 'c', 'n', 'c', 'c', '=', 'O', 'n', 'H', 'c', '14*', 'n', 'c']; link index list:(10, 9, 8, 7, 6, 20, 21, 22, 3, 1, 4, 5); link_bond:(1, 3)

***********jbonds:****************
((1, 3), ('5', '14'))
[1, 2, 0, -1]
---------------------------------------------------
link_idx:9
INPUT
frag smi:[9*]n1cnc2c(=O)[nH]c([14*])nc21;unique_atom_idx:(10, 9, 8, 7, 6, 20, 21, 22, 3, 1, 4, 5);link_value:9;bond_lst:(9, 10)
atom num==len(new_smi_char_lst):False
matched bond_pos:0
-----------------------------------

In [153]:
order_dict

{'[16*]c1ccc2ccccc2c1': 0,
 '[5*]N(C)C': 1,
 '[9*]n1cnc2c(=O)[nH]c([14*])nc21': 2}

In [154]:
jbond_lst,sorted(res_bond)

([[1, 2, 0, -1], [2, 0, 0, 0]],
 [((1, 3), ('5', '14')), ((9, 10), ('9', '16'))])